# **DeepLandforms - v2**

Author: giacomo.nodjoumi@hyranet.info - g.nodjoumi@jacobs-university.de

## DeepLandforms

With this notebook, users can use custom [YOLOv8](https://github.com/ultralytics/ultralytics) trained models for object detection and instance segmentation models on custom dataset of georeferenced images.
Results can be visualized directly in the noteboo using leafmap and WMS backend.

The output consist of a folder containing:
* Crop of the detections (georeferenced)
* Label file in COCO json format for each image for segmentation
* Label file in YOLO txt format for object detection
* Geopackage containing a single layer with image name, confidence leve, class.

## Usage

* Put or link the dataset into the **DeepLandforms** *.env* file
* Run docker-compose up
* Edit the *configs* section by editing the following parameters:

## Parameters
 ------------------------------------------------------------------
| **Parameter** | **Description** | **Example** |
| ---- | ---- | ---- |
| **data_dir** | local path of the data dir |  | /home/user/data |
| **model_path** | local path and name of the model  | /home/user/data/best.pt |
| **sam_checkpoint** | Segment Anything checkpoint for instance segmentation | /home/user/data/sam_vit_h_4b8939.pth |
| **dst_crs** | CRS of the final geopackage | provide as WKT or proj4 |
| **device** | device where to run the model | cuda or cpu |
------------------------------------------------------------------
Then just execute the notebook and monitor the training in **Tensorboard** container.

## Funding
*This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.*

In [57]:
from pyproj.crs import CRS
dst_crs = CRS.from_wkt('GEOGCRS["GCS_Mars_2000",DATUM["D_Mars_2000",ELLIPSOID["Mars_2000_IAU_IAG",3396190,169.894447223612,LENGTHUNIT["metre",1]]],PRIMEM["Reference_Meridian",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Not known."],AREA["World."],BBOX[-90,-180,90,180]],ID["ESRI",104905]]')
dst_crs = CRS.from_wkt('PROJCS["Moon_2000_Equidistant_Cylindrical",GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equidistant_Cylindrical_Ellipsoidal"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]')
dst_crs = CRS.from_wkt('GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]]')

In [2]:
from ultralytics import YOLO
import geopandas as gpd
import json
import math
import numpy as np
import os
import pandas as pd
import rasterio as rio
from utils.utils import get_paths, window_calc, mask2shape, bboxes2df, bbox2points, box2geotiff, box2sam, PlotMap

In [3]:
test_dir = '/home/Giacomo/data/lunar_craters/'
dst_dir = f"{test_dir}/detections"
os.makedirs(dst_dir, exist_ok=True)

In [125]:
model_path= '/home/Giacomo/DeepLandforms/YOLO/runs/detect/yolov8n_640_4_lunar_crater_combined_21082023_dsk_4/weights/best.pt'
model = YOLO(model_path)

In [126]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
SamAutomaticMaskGenerator, SamPredictor
sam_checkpoint = "/home/Giacomo/DeepLandforms/YOLO/runs/detect/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device="cuda")
predictor = SamPredictor(sam)

In [8]:
image_list = get_paths(test_dir, 'tiff')
image_list

['WAC_100m_Archytas_Crop-cog.tiff']

In [127]:
cols = []#'Image','Class','Conf']
geo_shape = gpd.GeoDataFrame(columns=cols)#, crs=img.crs)
geo_points = gpd.GeoDataFrame(columns=cols)#, crs=dst_crs)

In [8]:
#image_list

geo_points=gpd.read_file('/home/Giacomo/data/BC_n_SQCRP_n_CellSize_5_m_LIM_n_None_px_cog_n/detections/point_detections.gpkg')
geo_shape = gpd.read_file('/home/Giacomo/data/BC_n_SQCRP_n_CellSize_5_m_LIM_n_None_px_cog_n/detections/shape_detections.gpkg')

for file in image_list:
    try:
        image=f"{test_dir}/{file}"
        image_name, ext = os.path.splitext(os.path.basename(image))
        
        image_dir = os.path.dirname(image)
        img = rio.open(image)    
        aff = img.transform
        width = img.width
        height = img.height
        img_crs = img.crs
        results = model.predict(image, project="project",name="prediction", imgsz=640, conf=0.2)#, iou=0.3)#, half=True)  # predict on an imagesave_crop=True, save_txt=True, save_conf=True, 
        for result in results:#
            bboxes = np.array(result.boxes.data.cpu())
            xywh_bboxes = np.array(result.boxes.xywhn.data.cpu())
            classes = np.array(result.boxes.cls.data.cpu()).astype(int)
            confs = np.array(result.boxes.conf.data.cpu())
            shapes=[]
            for i, data in enumerate(bboxes):    
                bbox = data[0:4]
                conf = round(data[4],2)
                cls = result.names[data[5]]
                dst_name = box2geotiff(bbox, img,dst_dir, image_name, ext, cls, i)
    
                sam_gdf, shape_dict = box2sam(predictor, image, bbox, img_crs, conf, cls)
                shapes.append(shape_dict)
                if sam_gdf.crs != dst_crs:            
                    sam_gdf.to_crs(dst_crs, inplace=True)
                geo_shape=pd.concat([sam_gdf, geo_shape])
            
            yolo_fdf = bboxes2df(xywh_bboxes, classes,confs, cols=['x','y','w','h'])
            yolo_sdf=yolo_fdf[['Class','x','y','w','h']]#,'Conf']]
            csv_name = f"{dst_dir}/{image_name}.txt"
            yolo_sdf.to_csv(csv_name, header=False, sep=' ', index=False)        
            yolo_gdf = bbox2points(yolo_fdf, image)#xywh_bboxes, image_name,width, height,image)        
            if yolo_gdf.crs != dst_crs:            
                yolo_gdf.to_crs(dst_crs, inplace=True)
            geo_points=pd.concat([yolo_gdf, geo_points])
        
        label_dict = {
          "version": "5.2.1",
          "flags": {},
          "shapes": shapes,
          "imagePath": os.path.basename(image),
          "imageData": None,
          "imageHeight": height,
          "imageWidth": width
        }
        
        json_name = f"{dst_dir}/{image_name}.json"
        out_file = open(json_name, 'w')
        json.dump(label_dict,out_file,indent=2)
        out_file.close()      
    except Exception as e:
        print(e)
            
        
geo_points.reset_index(drop=True)
geo_points.crs=dst_crs
point_gpkg = f"{dst_dir}/point_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geo_points.to_file(point_gpkg, layer='PointDetections', driver="GPKG")


geo_shape.reset_index(drop=True)
geo_shape.crs=dst_crs
shape_gpkg = f"{dst_dir}/shape_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geo_shape.to_file(shape_gpkg, layer='ShapesDetections', driver="GPKG")

map_select = PlotMap('Moon')
#geo_points.crs = "EPSG:4326"
#map_select.add_gdf(geo_points,layer_name='PointsDetections')

map_select

aaa

moon2000=CRS.from_wkt('GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]]')


geo_points_crs = geo_points.copy()
#geo_points.crs = dst_crs#"EPSG:4326"
geo_points_crs=geo_points_crs.to_crs(dst_crs)
#geo_points_crs
geo_points_crs.crs ="EPSG:4326"
geo_points_crs.crs

geo_shape_crs = geo_shape.copy()
#geo_shape_crs= geo_shape_crs.to_crs(moon2000)
geo_shape_crs.crs = "EPSG:4326"

map_select.add_gdf(geo_shape_crs,layer_name='Detections',fill_colors=["red", "green", "blue"])#,style = style,)
map_select.add_gdf(geo_points_crs,layer_name='PointsDetections',fill_colors=["red", "green", "blue"])#,style = style,)
map_select


map_select

geo_shape_crs = geo_shape.copy()

geo_shape_crs.crs = "EPSG:4326"
geo_shape= geo_shape.to_crs(moon2000)
map_select.add_gdf(geo_shape_crs,layer_name='Detections',fill_colors=["red", "green", "blue"])#,style = style,)
map_select

geo_shape_crs

In [128]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, visualize_object_predictions
from sahi.utils.cv import read_image_as_pil

In [129]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=model_path,
    
    confidence_threshold=0.25,
    device="cuda:0", # or 'cpu'
)

In [11]:
#image_list

image=f"{test_dir}/{image_list[0]}"
#for image in image_list:
print(f"{image}")
result = get_sliced_prediction(
    image,
    detection_model,
    slice_height = 128,
    slice_width = 128,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)

import math
sahibboxes = []
for sahibbox in result.object_prediction_list:
    points = [[math.floor(sahibbox.bbox.minx),math.floor(sahibbox.bbox.miny)],[math.ceil(sahibbox.bbox.maxx),math.ceil(sahibbox.bbox.maxy)]]
    shape_dict  = {
      "label": "1",
      "points": points,
      "group_id": None,
      "description": "",
      "shape_type": "rectangle",
      "flags": {}
    }
    sahibboxes.append(shape_dict)
image_name = os.path.splitext(os.path.basename(image))[0]#.split('.')[0]
sahiimg = rio.open(image)
sahiimg.shape
label_dict = {
          "version": "5.2.1",
          "flags": {},
          "shapes": sahibboxes,
          "imagePath": os.path.basename(image),
          "imageData": None,
          "imageHeight": sahiimg.height,
          "imageWidth": sahiimg.width
        }
        
json_name = f"{dst_dir}/{image_name}.json"
out_file = open(json_name, 'w')
json.dump(label_dict,out_file,indent=2)
out_file.close()      


image_list

box2sam(predictor, image, np.ceil(np.array(sahibbox.bbox.to_xyxy())), img_crs, conf, cls)

In [131]:
slice_h, slice_w = 320, 320
for file in image_list:
    try:
        image=f"{test_dir}/{file}"
        image_name, ext = os.path.splitext(os.path.basename(image))
        
        image_dir = os.path.dirname(image)
        img = rio.open(image)    
        aff = img.transform
        width = img.width
        height = img.height
        img_crs = img.crs
        result = get_sliced_prediction(
                                        image,
                                        detection_model,
                                        slice_height = slice_h,
                                        slice_width = slice_w,
                                        overlap_height_ratio = 0.25,
                                        overlap_width_ratio = 0.25
                                    )
        import math
        sahibboxes = []
        shapes=[]
        for i, sahibbox in enumerate(result.object_prediction_list):
            points = [[math.floor(sahibbox.bbox.minx),math.floor(sahibbox.bbox.miny)],[math.ceil(sahibbox.bbox.maxx),math.ceil(sahibbox.bbox.maxy)]]
            conf=sahibbox.score.value
            classes=sahibbox.category.name
            shape_dict  = {
              "label": "1",
              "points": points,
              "group_id": None,
              "description": "",
              "shape_type": "rectangle",
              "flags": {}
            }
            
            sahibboxes.append(shape_dict)
    
        
        #for result in results:#            
    
            bbox = np.array([math.floor(sahibbox.bbox.minx),math.floor(sahibbox.bbox.miny),math.ceil(sahibbox.bbox.maxx),math.ceil(sahibbox.bbox.maxy)])
            conf = sahibbox.score.value
            cls = sahibbox.category.name
            
            x = bbox[0]
            y = bbox[1]
            w = bbox[2]-bbox[0]
            h = bbox[3]-bbox[1]
            
            # Finding midpoints
            x_centre = (x + (x+w))/2
            y_centre = (y + (y+h))/2
            
            # Normalization
            x_centre = x_centre / slice_w
            y_centre = y_centre / slice_h
            w = w / slice_w
            h = h / slice_h
            
            # Limiting upto fix number of decimal places
            #x_centre = format(x_centre, '.6f')
            #y_centre = format(y_centre, '.6f')
            #w = format(w, '.6f')
            #h = format(h, '.6f')
            xywh_bboxes = [x_centre,y_centre,w,h]
            dst_name = box2geotiff(bbox, img,dst_dir, image_name, ext, cls, i)
    
            sam_gdf, shape_dict = box2sam(predictor, image, bbox, img_crs, conf, cls)
            shapes.append(shape_dict)
            if sam_gdf.crs != dst_crs:            
                sam_gdf.to_crs(dst_crs, inplace=True)
            geo_shape=pd.concat([sam_gdf, geo_shape])
            yolo_fdf = pd.DataFrame([xywh_bboxes])#, columns=['x','y','w','h','Class','conf'])
            yolo_fdf['Class']=classes
            yolo_fdf['Conf']=conf
            yolo_fdf.columns=['x','y','w','h','Class','conf']
            yolo_sdf=yolo_fdf[['Class','x','y','w','h']]#,'Conf']]        
            image_det_dir = f"{dst_dir}/{image_name}"
            csv_name = f"{image_det_dir}/{image_name}_ID{i}_{cls}.txt"
            yolo_sdf.to_csv(csv_name, header=False, sep=' ', index=False)        
            yolo_gdf = bbox2points(yolo_fdf, image)#xywh_bboxes, image_name,width, height,image)        
            if yolo_gdf.crs != dst_crs:            
                yolo_gdf.to_crs(dst_crs, inplace=True)
            geo_points=pd.concat([yolo_gdf, geo_points])
        
        label_dict = {
          "version": "5.2.1",
          "flags": {},
          "shapes": shapes,
          "imagePath": os.path.basename(image),
          "imageData": None,
          "imageHeight": height,
          "imageWidth": width
        }
        
        json_name = f"{os.path.splitext(dst_name)[0]}.json"
        out_file = open(json_name, 'w')
        json.dump(label_dict,out_file,indent=2)
        out_file.close() 


        image_name = os.path.splitext(os.path.basename(image))[0]#.split('.')[0]
        label_dict = {
                  "version": "5.2.1",
                  "flags": {},
                  "shapes": sahibboxes,
                  "imagePath": os.path.basename(image),
                  "imageData": None,
                  "imageHeight": height,
                  "imageWidth": width
                }
                
        json_name = f"{dst_dir}/{image_name}.json"
        out_file = open(json_name, 'w')
        json.dump(label_dict,out_file,indent=2)
        out_file.close()    
    except Exception as e:
        print(e)
        pass
            
        

        


Performing prediction on 225 number of slices.
local variable 'shape_list' referenced before assignment
cannot unpack non-iterable NoneType object


In [63]:
img_crs

CRS.from_wkt('GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')

In [95]:
del geo_points

In [135]:
geo_points.reset_index(drop=True)
#geo_points=geo_points.to_crs(dst_crs)
point_gpkg = f"{dst_dir}/point_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geo_points.to_file(point_gpkg, layer='PointDetections', driver="GPKG")


geo_shape.reset_index(drop=True)
#geo_shape = geo_shape.to_crs(dst_crs)
shape_gpkg = f"{dst_dir}/shape_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geo_shape.to_file(shape_gpkg, layer='ShapesDetections', driver="GPKG")

In [100]:
geo_points.reset_index(drop=True,inplace=True)

In [99]:
geo_points.loc[0]

,x,y,w,h,Class,conf,long,lat,geometry
0,13.628906,7.466797,0.031250,0.097656,0,0.503278,261.208576,4.840058,POINT (261.20858 4.84006)
0,18.701172,5.615234,0.058594,0.058594,0,0.505491,362.257953,18.016939,POINT (362.25795 18.01694)
0,17.919922,7.171875,0.128906,0.117188,0,0.508030,346.693937,6.938907,POINT (346.69394 6.93891)
0,21.654297,3.632812,0.089844,0.085938,0,0.513778,421.089935,32.125098,POINT (421.08994 32.12510)
0,11.259766,5.652344,0.097656,0.093750,0,0.516025,214.010696,17.752846,POINT (214.01070 17.75285)
0,8.857422,0.974609,0.082031,0.050781,0,0.530730,166.151346,51.042540,POINT (166.15135 51.04254)
0,2.449219,0.341797,0.046875,0.050781,0,0.531557,38.487501,55.546031,POINT (38.48750 55.54603)
0,11.136719,7.402344,0.320312,0.312500,0,0.543773,211.559364,5.298747,POINT (211.55936 5.29875)
0,6.570312,2.429688,0.070312,0.054688,0,0.583271,120.587688,40.687291,POINT (120.58769 40.68729)
0,14.552734,4.824219,0.105469,0.039062,0,0.607086,279.613026,23.646303,POINT (279.61303 23.64630)


In [94]:
del geo_shape

In [101]:
geo_shape.reset_index(inplace=True)

In [102]:
list(geo_shape.loc[0].geometry.exterior.coords)

[(1.1574223566637265, 51.71597189127862),
 (1.187102454261245, 51.71597189127862),
 (1.187102454261245, 51.71267410265667),
 (1.1969958201270838, 51.71267410265667),
 (1.1969958201270838, 51.70937631403473),
 (1.2035913973709764, 51.70937631403473),
 (1.2035913973709764, 51.66650506194942),
 (1.2068891859929227, 51.66650506194942),
 (1.2068891859929227, 51.646718330217745),
 (1.2002936087490301, 51.646718330217745),
 (1.2002936087490301, 51.643420541595795),
 (1.1936980315051375, 51.643420541595795),
 (1.1936980315051375, 51.64012275297385),
 (1.187102454261245, 51.64012275297385),
 (1.187102454261245, 51.6368249643519),
 (1.1706135111515135, 51.6368249643519),
 (1.1706135111515135, 51.64012275297385),
 (1.1574223566637265, 51.64012275297385),
 (1.1574223566637265, 51.643420541595795),
 (1.150826779419834, 51.643420541595795),
 (1.150826779419834, 51.646718330217745),
 (1.1442312021759413, 51.646718330217745),
 (1.1442312021759413, 51.65001611883969),
 (1.140933413553995, 51.6500161188

In [71]:
geo_shape.loc[0].geo

index                                                       0
Image                              WAC_100m_Archytas_Crop-cog
Class                                                       0
Conf                                                 0.502259
geometry    POLYGON ((59396.9237118162 1637001.5381902473,...
Name: 0, dtype: object

In [15]:
aaa
geo_shape.crs = 'EPSG:4326'

NameError: name 'aaa' is not defined

In [136]:
geos = geo_shape.copy()
#geos = geos.to_crs(dst_crs)

In [134]:
shape_gpkg = f"{dst_dir}/shape_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geos.to_file(shape_gpkg, layer='ShapesDetections', driver="GPKG")

In [82]:
def PlotMap(body):
    from localtileserver import TileClient, get_leaflet_tile_layer, examples
    from ipyleaflet import Map
    import rasterio as rio
    import leafmap
    from ipyleaflet import Map, WMSLayer, basemaps
    from owslib.wms import WebMapService
    from folium.plugins import Fullscreen, MeasureControl
    
    #import leafmap.foliumap as leafmap
    wms_url='https://explore.hyranet.info/geoserver/ows?service=WMS'
    layers='Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013-cog',
    wms = WebMapService(
        url=wms_url,
        version='1.3.0'
    )
    
    mars_projection = {
        "name": "EPSG:104905",
        "custom": True,  # This is important, it tells ipyleaflet that this projection is not on the predefined ones.
        #"proj4def": "+proj=longlat +a=3396190 +rf=169.894447223612 +no_defs +type=crs",
        "proj4def": "+proj=longlat +a=3396190 +b=3396190  +no_defs +type=crs",
        "origin": [0, 0],
        "bounds": [[-180, -90], [180, 90]],
        "resolutions": [512],
    }
    moon_projection = {
        "name": "EPSG:104903",
        "custom": True,  # This is important, it tells ipyleaflet that this projection is not on the predefined ones.
        #"proj4def": "+proj=longlat +a=3396190 +rf=169.894447223612 +no_defs +type=crs",
        "proj4def": "+proj=longlat +R=1737400  +no_defs +type=crs",
        "origin": [0, 0],
        "bounds": [[-180, -90], [180, 90]],
        "resolutions": [512],
    }
    wms_layers = list(wms.contents)
    
    if body=='Mars':
        baselayer='Mars:Mars_Viking_MDIM21_ClrMosaic_global_232m-cog'
        my_projection=mars_projection
    if body=='Moon':
        baselayer='Moon:Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013-cog'
        my_projection=moon_projection
    idx = wms_layers.index(baselayer)
    wms_layer = WMSLayer(url=wms_url, layers=wms_layers[idx], format='image/png', max_zoom=14, min_zoom=2, tiles=True)
    # Create a TileClient from a raster file
    
    
    
    map_select = leafmap.Map(
        basemap=wms_layer,
        tiles=True,    
        max_zoom=14,
        min_zoom=2,    
    )
    style = {
        "color": "#93c47d ",
        "weight": 5,
        "opacity": 100,
        "fill": True,
    }
    #map_select.set_view([(image.bounds.left, image.bounds.bottom), (image.bounds.right, image.bounds.top)])
    
    return(map_select)

In [43]:
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from ipyleaflet import Map
import rasterio as rio
import leafmap
from ipyleaflet import Map, WMSLayer, basemaps
from owslib.wms import WebMapService
from folium.plugins import Fullscreen, MeasureControl

#import leafmap.foliumap as leafmap
wms_url='https://explore.hyranet.info/geoserver/ows?service=WMS'
layers='Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013-cog',
wms = WebMapService(
url=wms_url,
version='1.3.0'
)
mars_projection = {
    "name": "EPSG:104905",
    "custom": True,  # This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    #"proj4def": "+proj=longlat +a=3396190 +rf=169.894447223612 +no_defs +type=crs",
    "proj4def": "+proj=longlat +a=3396190 +b=3396190  +no_defs +type=crs",
    "origin": [0, 0],
    "bounds": [[-180, -90], [180, 90]],
    "resolutions": [8192.0, 4096.0, 2048.0, 1024.0, 512.0, 256.0],
}

In [86]:
wms_layers = list(wms.contents)
wms_layers

['Moon:Lunar_Clementine_UVVIS_Warp_ClrRatio_Global_200m_Copernicus-cog',
 'Moon:Lunar_Kaguya_MIMap_ClinoPyroxenePercent_Copernicus_cog',
 'Moon:Lunar_Kaguya_MIMap_OlivinePercent_Copernicus_cog',
 'Moon:Lunar_Kaguya_MIMap_OrthoPyroxenePercent_Copernicus_cog',
 'Moon:Lunar_Kaguya_MIMap_PlagioclasePercent_Copernicus_cog',
 'Moon:Lunar_LRO_LOLAKaguya_DEMmerge_60N60S_59m_Copernicus-cog',
 'Moon:Lunar_LRO_LOLAKaguya_DEMmerge_60N60S_59m_hillshade_Copernicus-cog',
 'Moon:Lunar_LRO_LOLAKaguya_DEMmerge_60N60S_59m_slope_Copernicus-cog',
 'Moon:Lunar_LRO_LOLA_Global_LDEM_118m_Mar2014-cog',
 'Moon:Lunar_LRO_LROC-WAC_Mosaic_global_100m_Copernicus-cog',
 'Moon:Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013-cog',
 'Moon:Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013_104903-cog',
 'Moon:M3_Aristarcus',
 'Moon:M3_RGB_Aristarcus_Olivine_parameter_Chromite_parameter_Titanium_parameter',
 'Moon:M3_RGB_Aristarcus_Olivine_parameter_Chromite_parameter_Titanium_parameter_cog',
 'Moon:M3_RGB_Aristarcus_Reflecta

In [138]:
map_select = PlotMap('Moon')
geos.reset_index(inplace=True)
geos.crs = "EPSG:4326"
map_select.add_gdf(geos,layer_name='PointsDetections')

map_select

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…